<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Insights/blob/main/05-CSV_Data_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 2: Interacting with a CSV Data

## Setup and connect to the Azure OpenAI endpoint

**Note**: The pre-configured cloud resource grants you access to the Azure OpenAI GPT model. The key and endpoint provided below are intended for teaching purposes only. Your notebook environment is already set up with the necessary keys, which may differ from those used by the instructor during the filming.

In [ ]:
!pip install -r requirements.txt

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=<FILEID>'  -O ./.env

In [3]:
import os
import openai
import sys

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [9]:
import os
import pandas as pd

from IPython.display import Markdown, HTML, display
from langchain.schema import HumanMessage
from langchain.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.7, max_tokens=4096)


## Load the dataset

In [4]:
import os
import pandas as pd

!os.makedirs("data",exist_ok=True)
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)

/bin/bash: -c: line 1: syntax error near unexpected token `"data",exist_ok=True'
/bin/bash: -c: line 1: `os.makedirs("data",exist_ok=True)'
--2024-06-15 02:56:32--  https://covidtracking.com/data/download/all-states-history.csv
Resolving covidtracking.com (covidtracking.com)... 104.21.64.114, 172.67.183.132, 2606:4700:3032::ac43:b784, ...
Connecting to covidtracking.com (covidtracking.com)|104.21.64.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./data/all-states-history.csv’

all-states-history.     [ <=>                ]   2.61M  --.-KB/s    in 0.04s   

2024-06-15 02:56:33 (59.7 MB/s) - ‘./data/all-states-history.csv’ saved [2738601]



## Prepare the Langchain dataframe agent

In [7]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm=llm,df=df,verbose=True)

agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)
Observation: 20780
Thought: I now know the final answer
Final Answer: There are 20780 rows in the dataframe.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 20780 rows in the dataframe.'}

## Design your prompt and ask your question

In [10]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

QUESTION = "How may patients were hospitalized during July 2020"
"in Texas, and nationwide as the total of all states?"
"Use the hospitalizedIncrease column"


agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...
Thought: I should set the pandas display options to show all columns first.
Action: python_repl_ast
Action Input: pd.set_option('display.max_columns', None)
Observation: NameError: name 'pd' is not defined
Thought: I need to import pandas first.
Action: python_repl_ast
Action Input: import pandas as pd
Observation: 
Thought: I imported pandas, now I can set the display options.
Action: python_repl_ast
Action Input: pd.set_option('display.max_columns', None)
Observation: 
Thought: I can now get the column names.
Action: python_repl_ast
Action Input: df.columns
Observation: Index(['date', 'state', 'death', 'deathConfirmed', 'deathIncrease',
       'deathProbable', 'hospitalized', 'hospitalizedCumulative',
       'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative',
       'inIcuCurrently', 'negative', 'negativeIncrease',
       'negativeTestsAntibody', 'negativeTestsPeopleAntibody',
       'negativeTestsViral', 'onVentilatorCumulative

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nHow may patients were hospitalized during July 2020\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how you got\nto the answer on a section that starts with: "\n\nExplanation:\n".\nIn the explanation, 